In [10]:
import gymnasium
import flappy_bird_gymnasium
import numpy as np
import pygame
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from enum import IntEnum
from flappy_bird_gymnasium.envs.flappy_bird_env import FlappyBirdEnv
from flappy_bird_gymnasium.envs.flappy_bird_env import Actions
from flappy_bird_gymnasium.envs.lidar import LIDAR
from flappy_bird_gymnasium.envs.constants import (
    PLAYER_FLAP_ACC,
    PLAYER_ACC_Y,
    PLAYER_MAX_VEL_Y,
    PLAYER_HEIGHT,
    PLAYER_VEL_ROT,
    PLAYER_WIDTH,
    PIPE_WIDTH,
    PIPE_VEL_X,
)



In [25]:

# Neural Network Model for Q-value approximation
class DQNCNN(nn.Module):
    def __init__(self, input_dim, action_space):
        super(DQNCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)  # 1 input channel
        self.conv2 = nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * input_dim, 128)
        self.fc2 = nn.Linear(128, action_space)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension: (batch_size, 1, input_dim)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


# Replay Memory to store experiences
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


# DQN Agent Class
class DQNAgent:
    def __init__(self, env):
        # Environment
        self.env = env
        self.state_dim = env.observation_space.shape[0]  # First dimension of observation
        self.action_space = env.action_space.n

        # Hyperparameters
        self.learning_rate = 0.001
        self.discount_factor = 0.99
        self.epsilon = 1.0  # Initial exploration rate
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.batch_size = 64
        self.memory_size = 10000
        self.episodes = 10000
        self.target_update_freq = 10

        # Initialize policy and target networks
        self.policy_net = DQNCNN(self.state_dim, self.action_space)
        self.target_net = DQNCNN(self.state_dim, self.action_space)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        # Optimizer and Replay Memory
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.memory = ReplayMemory(self.memory_size)

    def select_action(self, state, testing=False):
        """Epsilon-greedy action selection."""
        if not testing and random.random() < self.epsilon:
            return random.randint(0, self.action_space - 1)  # Random action
        else:
            with torch.no_grad():
                state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
                return self.policy_net(state).argmax(dim=1).item()

    def optimize_model(self):
        """Sample a batch from memory and optimize the policy network."""
        if len(self.memory) < self.batch_size:
            return

        batch = self.memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # Convert to tensors
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.long).unsqueeze(1)
        rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1)

        # Compute Q-values and targets
        q_values = self.policy_net(states).gather(1, actions)
        next_q_values = self.target_net(next_states).max(1, keepdim=True)[0]
        targets = rewards + (self.discount_factor * next_q_values * (1 - dones))

        # Loss and backpropagation
        loss = nn.MSELoss()(q_values, targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def train(self):
        """Train the agent."""
        for episode in range(self.episodes):
            state, _ = self.env.reset()
            done = False
            total_reward = 0

            while not done:
                # Select and execute action
                action = self.select_action(state)
                next_state, reward, done, _, _ = self.env.step(action)
                self.memory.push(state, action, reward, next_state, done)
                state = next_state
                total_reward += reward

                # Optimize model
                self.optimize_model()

            # Update target network periodically
            if episode % self.target_update_freq == 0:
                self.target_net.load_state_dict(self.policy_net.state_dict())

            # Decay epsilon
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {self.epsilon:.4f}")

        self.env.close()
        print("Training complete!")

    def test(self, num_episodes=10):
        """Test the trained policy with real-time rendering."""
        print("\nTesting the trained policy...\n")
        self.epsilon = 0.0  # Disable exploration
        test_env = gymnasium.make("FlappyBird-v0", render_mode="human",use_lidar=True)  # Render in "human" mode 
        total_rewards = []

        for episode in range(num_episodes):
            state, _ = test_env.reset()
            done = False
            total_reward = 0

            while not done:
                action = self.select_action(state, testing=True)
                next_state, reward, done, _, _ = test_env.step(action)
                state = next_state
                total_reward += reward

            total_rewards.append(total_reward)
            print(f"Test Episode: {episode + 1}, Total Reward: {total_reward}")

        avg_reward = np.mean(total_rewards)
        print(f"\nAverage Reward over {num_episodes} Test Episodes: {avg_reward}")
        test_env.close()

# Policy Gradient Neural Network Model
class PolicyGradientCNN(nn.Module):
    def __init__(self, input_dim, action_space):
        super(PolicyGradientCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)  # 1 input channel
        self.conv2 = nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * input_dim, 128)
        self.fc2 = nn.Linear(128, action_space)  # Outputs logits for actions

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension: (batch_size, 1, input_dim)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        return torch.softmax(self.fc2(x), dim=-1)  # Apply softmax for probabilities


# Policy Gradient Agent Class
class PolicyGradientAgent:
    def __init__(self, env):
        # Environment
        self.env = env
        self.state_dim = env.observation_space.shape[0]  # First dimension of observation
        self.action_space = env.action_space.n

        # Hyperparameters
        self.learning_rate = 0.001
        self.discount_factor = 0.99
        self.episodes = 10000

        # Initialize policy network
        self.policy_net = PolicyGradientCNN(self.state_dim, self.action_space)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)

        # Memory for rewards and log probabilities
        self.log_probs = []
        self.rewards = []

    def select_action(self, state):
        """Select an action based on policy."""
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        probs = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        self.log_probs.append(action_dist.log_prob(action))
        return action.item()

    def compute_returns(self):
        """Compute discounted returns for each time step."""
        returns = []
        G = 0
        for reward in reversed(self.rewards):
            G = reward + self.discount_factor * G
            returns.insert(0, G)
        returns = torch.tensor(returns, dtype=torch.float32)
        # Normalize returns for better convergence
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        return returns

    def train_step(self):
        """Perform a training step after one episode."""
        returns = self.compute_returns()
        loss = 0
        for log_prob, G in zip(self.log_probs, returns):
            loss += -log_prob * G  # Negative sign for gradient ascent

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Clear memory
        self.log_probs = []
        self.rewards = []

    def train(self):
        """Train the agent."""
        for episode in range(self.episodes):
            state, _ = self.env.reset()
            done = False
            total_reward = 0

            while not done:
                action = self.select_action(state)
                next_state, reward, done, _, _ = self.env.step(action)
                self.rewards.append(reward)
                state = next_state
                total_reward += reward

            # Perform a training step after the episode ends
            self.train_step()
            print(f"Episode: {episode + 1}, Total Reward: {total_reward}")

        self.env.close()
        print("Training complete!")

    def test(self, num_episodes=10):
        """Test the trained policy with real-time rendering."""
        print("\nTesting the trained policy...\n")
        test_env = gym.make("FlappyBird-v0", render_mode="human", use_lidar=True)
        total_rewards = []

        for episode in range(num_episodes):
            state, _ = test_env.reset()
            done = False
            total_reward = 0

            while not done:
                state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
                probs = self.policy_net(state)
                action = torch.argmax(probs).item()  # Select action with highest probability
                next_state, reward, done, _, _ = test_env.step(action)
                state = next_state
                total_reward += reward

            total_rewards.append(total_reward)
            print(f"Test Episode: {episode + 1}, Total Reward: {total_reward}")

        avg_reward = np.mean(total_rewards)
        print(f"\nAverage Reward over {num_episodes} Test Episodes: {avg_reward}")
        test_env.close()




In [26]:
env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array",use_lidar=True ) ##use_lidar=True option avaliable


agent = DQNAgent(env)
agent.train()
agent.test(num_episodes=10)


pg_agent = PolicyGradientAgent(env)
pg_agent.train()
pg_agent.test(num_episodes=10)


/home/cvdg/anaconda3/envs/flappy/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/cvdg/anaconda3/envs/flappy/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Episode: 1, Total Reward: -7.499999999999998, Epsilon: 0.9950
Episode: 2, Total Reward: -8.099999999999998, Epsilon: 0.9900
Episode: 3, Total Reward: -8.7, Epsilon: 0.9851
Episode: 4, Total Reward: -6.299999999999999, Epsilon: 0.9801
Episode: 5, Total Reward: -6.299999999999999, Epsilon: 0.9752
Episode: 6, Total Reward: -8.099999999999998, Epsilon: 0.9704
Episode: 7, Total Reward: -6.299999999999999, Epsilon: 0.9655
Episode: 8, Total Reward: -7.499999999999998, Epsilon: 0.9607
Episode: 9, Total Reward: -5.699999999999998, Epsilon: 0.9559
Episode: 10, Total Reward: -8.7, Epsilon: 0.9511
Episode: 11, Total Reward: -7.499999999999998, Epsilon: 0.9464
Episode: 12, Total Reward: -8.099999999999998, Epsilon: 0.9416
Episode: 13, Total Reward: -8.099999999999998, Epsilon: 0.9369
Episode: 14, Total Reward: -8.099999999999998, Epsilon: 0.9322
Episode: 15, Total Reward: -7.499999999999998, Epsilon: 0.9276
Episode: 16, Total Reward: -7.499999999999998, Epsilon: 0.9229
Episode: 17, Total Reward: -7

In [17]:
agent.test(num_episodes=10)


Testing the trained policy...

Test Episode: 1, Total Reward: -9.299999999999999
Test Episode: 2, Total Reward: -9.299999999999999
Test Episode: 3, Total Reward: -9.299999999999999
Test Episode: 4, Total Reward: -9.299999999999999
Test Episode: 5, Total Reward: -9.299999999999999
Test Episode: 6, Total Reward: -8.7
Test Episode: 7, Total Reward: -8.7
Test Episode: 8, Total Reward: -0.8999999999999986
Test Episode: 9, Total Reward: -9.299999999999999
Test Episode: 10, Total Reward: 2.0999999999999996

Average Reward over 10 Test Episodes: -7.2


In [24]:
def env_customization(update_step=0):

    def custom_step(self, action):

        self._sound_cache = None
        if action == Actions.FLAP:
            if self._player_y > -2 * PLAYER_HEIGHT:
                self._player_vel_y = PLAYER_FLAP_ACC
                self._player_flapped = True
                self._sound_cache = "wing"

        # check for score
        player_mid_pos = self._player_x + PLAYER_WIDTH / 2
        for pipe in self._upper_pipes:
            pipe_mid_pos = pipe["x"] + PIPE_WIDTH / 2
            if pipe_mid_pos <= player_mid_pos < pipe_mid_pos + 4:
                self._score += 1
                reward = 1  # reward for passed pipe
                self._sound_cache = "point"

        # player_index base_x change
        if (self._loop_iter + 1) % 3 == 0:
            self._player_idx = next(self._player_idx_gen)

        self._loop_iter = (self._loop_iter + 1) % 30
        self._ground["x"] = -((-self._ground["x"] + 100) % self._base_shift)

        # rotate the player
        if self._player_rot > -90:
            self._player_rot -= PLAYER_VEL_ROT

        # player's movement
        if self._player_vel_y < PLAYER_MAX_VEL_Y and not self._player_flapped:
            self._player_vel_y += PLAYER_ACC_Y

        if self._player_flapped:
            self._player_flapped = False

            # more rotation to cover the threshold
            # (calculated in visible rotation)
            self._player_rot = 45

        self._player_y += min(
            self._player_vel_y, self._ground["y"] - self._player_y - PLAYER_HEIGHT
        )

        # move pipes to left
        for up_pipe, low_pipe in zip(self._upper_pipes, self._lower_pipes):
            up_pipe["x"] += PIPE_VEL_X
            low_pipe["x"] += PIPE_VEL_X

            # it is out of the screen
            if up_pipe["x"] < -PIPE_WIDTH:
                new_up_pipe, new_low_pipe = self._get_random_pipe()
                up_pipe["x"] = new_up_pipe["x"]
                up_pipe["y"] = new_up_pipe["y"]
                low_pipe["x"] = new_low_pipe["x"]
                low_pipe["y"] = new_low_pipe["y"]

        if self.render_mode == "human":
            self.render()

        obs, reward_private_zone = self._get_observation()
        if reward is None:
            if reward_private_zone is not None:
                reward = reward_private_zone
            else:
                reward = 0.1  # reward for staying alive

        # check
        if self._debug and self._use_lidar:
            # sort pipes by the distance between pipe and agent
            up_pipe = sorted(
                self._upper_pipes,
                key=lambda x: np.sqrt(
                    (self._player_x - x["x"]) ** 2
                    + (self._player_y - (x["y"] + PIPE_HEIGHT)) ** 2
                ),
            )[0]
            # find ray closest to the obstacle
            min_index = np.argmin(obs)
            min_value = obs[min_index] * LIDAR_MAX_DISTANCE
            # mean approach to the obstacle
            if "pipe_mean_value" in self._statistics:
                self._statistics["pipe_mean_value"] = self._statistics[
                    "pipe_mean_value"
                ] * 0.99 + min_value * (1 - 0.99)
            else:
                self._statistics["pipe_mean_value"] = min_value

            # Nearest to the pipe
            if "pipe_min_value" in self._statistics:
                if min_value < self._statistics["pipe_min_value"]:
                    self._statistics["pipe_min_value"] = min_value
                    self._statistics["pipe_min_index"] = min_index
            else:
                self._statistics["pipe_min_value"] = min_value
                self._statistics["pipe_min_index"] = min_index

            # Nearest to the ground
            diff = np.abs(self._player_y - self._ground["y"])
            if "ground_min_value" in self._statistics:
                if diff < self._statistics["ground_min_value"]:
                    self._statistics["ground_min_value"] = diff
            else:
                self._statistics["ground_min_value"] = diff

        # agent touch the top of the screen as punishment
        if self._player_y < 0:
            reward = -0.5

        # check for crash
        if self._check_crash():
            self._sound_cache = "hit"
            reward = -1  # reward for dying
            terminal = True
            self._player_vel_y = 0
            if self._debug and self._use_lidar:
                if ((self._player_x + PLAYER_WIDTH) - up_pipe["x"]) > (0 + 5) and (
                    self._player_x - up_pipe["x"]
                ) < PIPE_WIDTH:
                    print("BETWEEN PIPES")
                elif ((self._player_x + PLAYER_WIDTH) - up_pipe["x"]) < (0 + 5):
                    print("IN FRONT OF")
                print(
                    f"obs: [{self._statistics['pipe_min_index']},"
                    f"{self._statistics['pipe_min_value']},"
                    f"{self._statistics['pipe_mean_value']}],"
                    f"Ground: {self._statistics['ground_min_value']}"
                )

        info = {"score": self._score}

        return (
            obs,
            reward,
            terminal,
            (self._score_limit is not None) and (self._score >= self._score_limit),
            info,
        )

    
    if update_step:    
        FlappyBirdEnv.step = custom_step

In [ ]:
def env_customization(update_step=0):

    def custom_step(self, action):


        self._sound_cache = None
        if action == Actions.FLAP:
            if self._player_y > -2 * PLAYER_HEIGHT:
                self._player_vel_y = PLAYER_FLAP_ACC
                self._player_flapped = True
                self._sound_cache = "wing"

        # Check for score
        player_mid_pos = self._player_x + PLAYER_WIDTH / 2
        for pipe in self._upper_pipes:
            pipe_mid_pos = pipe["x"] + PIPE_WIDTH / 2
            if pipe_mid_pos <= player_mid_pos < pipe_mid_pos + 4:
                self._score += 1
                reward = 1  # reward for passed pipe
                self._sound_cache = "point"

        # Player movement and rotation
        if self._player_rot > -90:
            self._player_rot -= PLAYER_VEL_ROT
        if self._player_vel_y < PLAYER_MAX_VEL_Y and not self._player_flapped:
            self._player_vel_y += PLAYER_ACC_Y
        if self._player_flapped:
            self._player_flapped = False
            self._player_rot = 45
        self._player_y += min(
            self._player_vel_y, self._ground["y"] - self._player_y - PLAYER_HEIGHT
        )

        # Move pipes
        for up_pipe, low_pipe in zip(self._upper_pipes, self._lower_pipes):
            up_pipe["x"] += PIPE_VEL_X
            low_pipe["x"] += PIPE_VEL_X
            if up_pipe["x"] < -PIPE_WIDTH:
                new_up_pipe, new_low_pipe = self._get_random_pipe()
                up_pipe.update(new_up_pipe)
                low_pipe.update(new_low_pipe)

        if self.render_mode == "human":
            self.render()

        obs, reward_private_zone = self._get_observation()
        if reward is None:
            if reward_private_zone is not None:
                reward = reward_private_zone
            else:
                reward = 0.1  # Change reward for staying alive

        # Agent touches the top of the screen
        if self._player_y < 0 or self._player_y + PLAYER_HEIGHT/2 > self._ground["y"]:
            reward = -1.0
            terminal = True

        # Check for crash
        if self._check_crash():
            self._sound_cache = "hit"
            reward = -1  # reward for dying
            terminal = True
            self._player_vel_y = 0

        info = {"score": self._score}

        return (
            obs,
            reward,
            terminal,
            (self._score_limit is not None) and (self._score >= self._score_limit),
            info,
        )

    
    if update_step:    
        FlappyBirdEnv.step = custom_step